# Implanatação de algumas Bibliotacas

In [ ]:
import csv
import numpy as np
import pandas as pd
import sys
from tkinter import Tk
from tkinter import filedialog
from datetime import date, datetime, timedelta
#import os
#from ciso8601 import parse_datetime

# Criando variaveis para os arquivos permitidos

In [ ]:
p_cmpid4  = pd.read_excel('PQ379.xlsx')      ### leitura do arquivo com os sinais dos cpmids do quadro 379
param4    = pd.read_excel('LQ379.xlsx')      ### leitura do arquivo com o layout atualizado do quadro 379

In [ ]:
control  = 0                                              ### controle de backup    
p_cmpid4['cmpid'] = p_cmpid4.cmpid.astype(str)            ### transforma o campo cmpid - QE 379 - do arq auxiliar para string

In [ ]:
## FUNÇÃO PARA SELEÇÃO DO ARQUIVO

class qe_s:

    def __init__(self):
        self.df = {nome[s]: [] for s in range(nome.shape[0])}
  
    def make_df(self):
        lines = []
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        for file in filedialog.askopenfilenames(filetypes=[("Arquivos de Text", "*.txt")]):
            for line in open(file, encoding="utf-8"):
                line = line.replace(",",".").strip()
                for x in range(nome.shape[0]):
                    self.df[nome[x]].append(line[i[x]:j[x]])
                    
        return pd.DataFrame(self.df)

In [ ]:
nome     = param4.Campo                      ### guarda os nomes das colunas existentes no quadro 379
i        = param4.Ind_inf                    ### guarda a posição no arquivo que inicia um determinado campo
j        = param4.Ind_sup                    ### guarda a posição no arquivo que termina um determinado campo

quadro = qe_s()       
df_main = quadro.make_df()
df_main

## SELECINE O(S) ARQUIVO(S) DESEJADO(S)

In [ ]:
###################################
###Importacao do Quadro379.xlsx ###
####que serão usados para fins#####
######### de comparação ###########
###################################

#Pasta CMPID
arq_cmpid = pd.read_excel('Quadro379.xlsx','CMPID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta RAMCODIGO
arq_ramcod = pd.read_excel('Quadro379.xlsx','RAMCODIGO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta ENTCODIGO
arq_entcod = pd.read_excel('Quadro379.xlsx','Cod_SUSEP',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MRFMESANO
arq_mrfmesano = pd.read_excel('Quadro379.xlsx','MRFMESANO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta rel_CMPID_TPMOID
arq_relacao = pd.read_excel('Quadro379.xlsx','rel_CMPID_TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta TPMOID
arq_tpmoid = pd.read_excel('Quadro379.xlsx','TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta TPMOPREV
arq_tpmoprev = pd.read_excel('Quadro379.xlsx','TPMOPREV',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

In [ ]:
# Criando uma lista para guardar todas as criticas impeditivas
criticas_impeditivas = []
criticas = []

# Criticas (Validacoes)

In [ ]:
#7395.3 Verifica se o campo sequencial EBISEQ é uma sequência válida, que se inicia em 0000001 
array3 = df_main['EBISEQ']
x = list(array3)                                     ##Dados do arquivo
arrayteste = x[:]
arrayteste.sort()                                    ##Dados sequenciados
if (arrayteste == x):                                ##Comparando
    print('Tudo certo, esta de forma sequencial.')
else: 
    print('ERRO!!!, Não prossiga.')
    critica7395_3 = str('Critica 7395.3, Não temos uma sequencia valida na coluna EBISEQ')
    criticas_impeditivas.append(critica7395_3)

In [ ]:
#7395.2 Verifica o tamanho padrão da linha (205 caracteres)
tamanho = 0
for i in range(0,df_main.shape[0]):                    #Passando por cada linha
    for iten in df_main.columns:                       #Passando por cada coluna
        tamanho = tamanho + len(df_main[iten][i])      #Contando Caracteres por linha
    if tamanho != 205:
        print('Erro na linha ',i,' o número de caracteres é de: ',tamanho)
        critica7395_2 = ("Critica 7395.2, Erro na linha ",i," o número de caracteres é de: ",tamanho)
        criticas_impeditivas.append(critica7395_2)
    tamanho = 0

In [ ]:
#7395.4 Verifica se o campo ENTCODIGO corresponde à sociedade que está enviando o FIP/SUSEP
entcod = set(arq_entcod['Cod_SUSEP'].astype('str'))                   #Escolhendo a coluna do arquivo modelo
dadosentcod = set(df_main['ENTCODIGO'])                               #Escolhendo a coluna dos dados
for itemtentcod in dadosentcod: 
    if itemtentcod not in entcod:
        print('ERRO!! ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        critica7395_4 = ('Critica 7395.4, ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        criticas_impeditivas.append(critica7395_4)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtentcod)
        continue

In [ ]:
#7395.5 Verifica se o campo MRFMESANO corresponde, respectivamente, ao ano, mês e último dia do mês de referência do FIP/SUSEP 
df_main.loc[:,'DTMRFMESANO'] = df_main['MRFMESANO'].astype('datetime64')     #Criando uma nova coluna no formato Date
mrfmesano = set(arq_mrfmesano['DATAS'].astype('datetime64'))                 #Escolhendo a coluna do arquivo modelo
dadosmrfmesano = set(df_main['DTMRFMESANO'])                                 #Escolhendo a coluna dos dados
for itemmrfmesano in dadosmrfmesano: 
    if itemmrfmesano not in mrfmesano:
        print('ERRO!! MRFMESANO não corresponde a uma operação valida', itemmrfmesano)
        critica7395_5 = ('Critica 7395.5, MRFMESANO não corresponde a uma Data valida', itemmrfmesano)
        criticas_impeditivas.append(critica7395_5)
    else:
        print('MRFMESANO corresponde a uma Data válida.', itemmrfmesano)
        continue

In [ ]:
#7395.6 Verifica se o campo QUAID corresponde ao quadro 379
array6 = set(df_main['QUAID'])           ##Verificando todos os Quadros que estamos tratando no documento.
for itemarray6 in array6:
    if itemarray6 == '379':
        print('Este se trata do Quadro:',itemarray6)
    else:
        print('ERRO!! Este se trata do Quadro:',itemarray6)
        critica7395_6 = ('Critica 7395.6, Este esta se tratando do(s) Quadro(s):',itemarray6)
        criticas_impeditivas.append(critica7395_6)

In [ ]:
#7395.7 Verifica se o campo TPMOID corresponde a um tipo de movimento valido 
tpmoid = set(arq_tpmoid['TPMOID'].astype('str'))                                    #Escolhendo a coluna do arquivo modelo
dadostpmoid = set(df_main['TPMOID'])                                                #Escolhendo a coluna dos dados
for itemtpmoid in dadostpmoid: 
    if itemtpmoid not in tpmoid:
        print('ERRO!! TPMOID não corresponde a uma operação valida', itemtpmoid)
        critica7395_7 = ('Critica 7395.7, TPMOID não corresponde a uma operação valida', itemtpmoid)
        criticas_impeditivas.append(critica7395_7)
    else:
        print('TPMOID corresponde a um tipo de operação válida.', itemtpmoid)
        continue

In [ ]:
#7395.8 Verifica se o campo TPMOPREV corresponde a um tipo de movimento de previdência valido 
tpmoprev = set(arq_tpmoprev['TPMOPREV'].astype('str'))                                  #Escolhendo a coluna do arquivo modelo
dadostpmoprev = set(df_main['TPMOPREV'])                                                #Escolhendo a coluna dos dados
for itemtpmoprev in dadostpmoprev: 
    if itemtpmoprev not in tpmoprev:
        print('ERRO!! TPMOPREV não corresponde a uma operação valida', itemtpmoprev)
        critica7395_8 = ('Critica 7395.8, TPMOPREV não corresponde a uma operação valida', itemtpmoprev)
        criticas_impeditivas.append(critica7395_8)
    else:
        print('TPMOPREV corresponde a um tipo de operação válida.', itemtpmoprev)
        continue

In [ ]:
#7395.9 Verifica se o campo CMPID corresponde a um tipo de operação válida
cmpid = set(arq_cmpid['CMPID'].astype('str'))             #Escolhendo a coluna do arquivo modelo
dadoscmpid = set(df_main['CMPID'])                                 #Escolhendo a coluna dos dados
for itemcmpid in dadoscmpid: 
    if itemcmpid not in cmpid:
        print('ERRO!! O campo CMPID não corresponde a um tipo de operação válida.', itemcmpid)
        critica73995_9 = ('Critica 7395.9, O campo CMPID não corresponde a um tipo de operação válida.', itemcmpid)
        criticas_impeditivas.append(critica7395_9)
    else:
        print('O campo CMPID corresponde a um tipo de operação válida.', itemcmpid)
        continue

In [ ]:
#7395.10 Verifica se o PLNCODIGO pertence ao Cadastro de Planos da entidade

In [ ]:
#7395.11 Verifica se o CPF do participante (EBICPFPART) e do beneficiário (EBICPFBENF) são inteiros e válidos, exceto para preenchimento com zeros

#Criação da função
import re

def isCpfValid(cpf):
    """ If cpf in the Brazilian format is valid, it returns True, otherwise, it returns False. """

    # Check if type is str
    if not isinstance(cpf,str):
        return False

    # Remove some unwanted characters
    cpf = re.sub("[^0-9]",'',cpf)
    
    # Verify if CPF number is equal
    if cpf=='00000000000' or cpf=='11111111111' or cpf=='22222222222' or cpf=='33333333333' or cpf=='44444444444' or cpf=='55555555555' or cpf=='66666666666' or cpf=='77777777777' or cpf=='88888888888' or cpf=='99999999999':
        return False

    # Checks if string has 11 characters
    if len(cpf) != 11:
        return False

    sum = 0
    weight = 10

    """ Calculating the first cpf check digit. """
    for n in range(9):
        sum = sum + int(cpf[n]) * weight

        # Decrement weight
        weight = weight - 1

    verifyingDigit = 11 -  sum % 11

    if verifyingDigit > 9 :
        firstVerifyingDigit = 0
    else:
        firstVerifyingDigit = verifyingDigit

    """ Calculating the second check digit of cpf. """
    sum = 0
    weight = 11
    for n in range(10):
        sum = sum + int(cpf[n]) * weight

        # Decrement weight
        weight = weight - 1

    verifyingDigit = 11 -  sum % 11

    if verifyingDigit > 9 :
        secondVerifyingDigit = 0
    else:
        secondVerifyingDigit = verifyingDigit

    if cpf[-2:] == "%s%s" % (firstVerifyingDigit,secondVerifyingDigit):
        return True
    return False

In [ ]:
# Realizando Check no Campo EBICPFPART
cpfparticipante = set(df_main['EBICPFPART'])

for itemcpfp in cpfparticipante:
    verificacao1 = isCpfValid(itemcpfp)
    if verificacao1 == False:
        print("Erro!! Critica 7395.11 o CPF ", itemcpfp, " não é um CPF valido.")
        critica7395_11 = ("Critica 7395.11","o CPF ", itemcpfp, " não é um CPF valido.")
        criticas_impeditivas.append(critica7395_11)

print('Demais CPF do participante nos campos (EBICPFPART) são validos')      

In [ ]:
# Realizando Check no Campo EBICPFBENF
cpfbeneficiario = set(df_main['EBICPFBENF'])

for itemcpfb in cpfbeneficiario:
    verificacao2 = isCpfValid(itemcpfb)
    if verificacao2 == False:
        print("Erro!! Critica 7395.11 o CPF ", itemcpfb, " não é um CPF valido.")
        critica7395_11 = ("Critica 7395.11","o CPF ", itemcpfb, " não é um CPF valido.")
        criticas_impeditivas.append(critica7395_11)

print('Demais CPF do participante nos campos (EBICPFPART) são validos')   

In [ ]:
#7395.12 Verifica se os campos EBIDATAINICIO, EBIDATAFIM, EBIDATAOCORR, EBIDATAREG e EBIDATACOMUNICA correspondem a uma data válida 
#e se estão compreendidos entre os anos de 1901 e 2099.

#####################################################
##Readicionando as colunas de Data de str para date##
#####################################################

df_main.loc[:,'DTEBIDATAINICIO'] = df_main['EBIDATAINICIO'].astype('datetime64')
df_main.loc[:,'DTEBIDATAFIM'] = df_main['EBIDATAFIM'].astype('datetime64')
df_main.loc[:,'DTEBIDATAOCORR'] = df_main['EBIDATAOCORR'].astype('datetime64')
df_main.loc[:,'DTEBIDATAREG'] = df_main['EBIDATAREG'].astype('datetime64')
df_main.loc[:,'DTEBIDATACOMUNICA'] = df_main['EBIDATACOMUNICA'].astype('datetime64')

#Adicionado colunas EBIDATAINICIO, EBIDATAFIM, EBIDATAOCORR, EBIDATAREG e EBIDATACOMUNICA no tipo Date

df_dates_comparacao = pd.DataFrame(df_main['DTMRFMESANO'] - timedelta(days = 43800))
df_dates_comparacao.loc[:,'DTMRFMESANO_max'] = df_main['DTMRFMESANO'] + timedelta(days = 29200)

In [ ]:
comparação_01 = (df_main['DTEBIDATAINICIO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBIDATAINICIO'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc01 = df_main.loc[comparação_01]
if len(dfc01) > 0:
    dfc01.loc[:,'ID da Critica'] = ('CRITICA 7395.12')
dfc01

In [ ]:
comparação_02 = (df_main['DTEBIDATAFIM'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBIDATAFIM'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc02 = df_main.loc[comparação_02]
if len(dfc02) > 0:
    dfc02.loc[:,'ID da Critica'] = ('CRITICA 7395.12')
dfc02

In [ ]:
comparação_03 = (df_main['DTEBIDATAOCORR'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBIDATAOCORR'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc03 = df_main.loc[comparação_03]
if len(dfc03) > 0:
    dfc03.loc[:,'ID da Critica'] = ('CRITICA 7395.12')
dfc03

In [ ]:
comparação_04 = (df_main['DTEBIDATAREG'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBIDATAREG'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc04 = df_main.loc[comparação_04]
if len(dfc04) > 0:
    dfc04.loc[:,'ID da Critica'] = ('CRITICA 7395.12')
dfc04

In [ ]:
comparação_05 = (df_main['DTEBIDATACOMUNICA'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTEBIDATACOMUNICA'] > df_dates_comparacao['DTMRFMESANO_max'])
dfc05 = df_main.loc[comparação_05]
if len(dfc05) > 0:
    dfc05.loc[:,'ID da Critica'] = ('CRITICA 7395.12')
dfc05

In [ ]:
#7395.13 Verifica se as datas relacionadas ao evento (EBIDATAOCORR, EBIDATAREG e EBIDATARCOMUNICA), 
#para a combinação de TPMOID 0001 e TPMOPREV 0001, estão entre os limites de trinta anos para mais ou menos em relação ao mês de referência
combinação_13 = (df_main['TPMOID'] == '0001') & (df_main['TPMOPREV'] == '0001')

df_dates_comparacao.loc[:,'DTMRFMESANO-30'] = df_main['DTMRFMESANO'] - timedelta(days = 10950)
df_dates_comparacao.loc[:,'DTMRFMESANO+30'] = df_main['DTMRFMESANO'] + timedelta(days = 10950)

In [ ]:
mask13_1 = (df_main['DTEBIDATAOCORR'] < df_dates_comparacao['DTMRFMESANO-30']) | (df_main['DTEBIDATAOCORR'] > df_dates_comparacao['DTMRFMESANO+30'])
df13_1 = df_main.loc[mask13_1]
if len(df13_1) > 0:
    df13_1.loc[:,'ID da Critica'] = ('CRITICA 7395.13')
df13_1

In [ ]:
mask13_2 = (df_main['DTEBIDATAREG'] < df_dates_comparacao['DTMRFMESANO-30']) | (df_main['DTEBIDATAREG'] > df_dates_comparacao['DTMRFMESANO+30'])
df13_2 = df_main.loc[mask13_2]
if len(df13_2) > 0:
    df13_2.loc[:,'ID da Critica'] = ('CRITICA 7395.13')
df13_2

In [ ]:
mask13_3 = (df_main['DTEBIDATACOMUNICA'] < df_dates_comparacao['DTMRFMESANO-30']) | (df_main['DTEBIDATACOMUNICA'] > df_dates_comparacao['DTMRFMESANO+30'])
df13_3 = df_main.loc[mask13_3]
if len(df13_3) > 0:
    df13_3.loc[:,'ID da Critica'] = ('CRITICA 7395.13')
df13_3

In [ ]:
#7395.14 Verifica se o campo EBIBENVEN é inteiro e maior ou igual a zero. 
df_main.loc[:,'iEBIBENVEN'] = df_main['EBIBENVEN'].astype('int64')
dadosebibenven = set(df_main['iEBIBENVEN'])
for itemebibenven in dadosebibenven:
    if itemebibenven < 0:
        print('Erro!! O campo EBIBENVEN não é maior ou igual a zero.',itemebibenven)
        critica7395_14 = ('Critica 7395.14, O campo EBIBENVEN não é maior ou igual a zero.',itemebibenven)
    else:
        print('O campo EBIBENVEN é inteiro e maior ou igual a zero.', itemebibenven)
        continue

In [ ]:
#7395.15 Verifica se os campos EBIVALORPBAC, EBIVALORPBC, EBIVALORMOV e EBIVALORMON são float

###########################################
##Readicionando as colunas str para float##
###########################################

df_main.loc[:,'fEBIVALORPBAC'] = df_main['EBIVALORPBAC'].astype('float64')
df_main.loc[:,'fEBIVALORPBC'] = df_main['EBIVALORPBC'].astype('float64')
df_main.loc[:,'fEBIVALORMOV'] = df_main['EBIVALORMOV'].astype('float64')
df_main.loc[:,'fEBIVALORMON'] = df_main['EBIVALORMON'].astype('float64')

#Adicionado coluna EBIVALORPBAC, EBIVALORPBC, EBIVALORMOV e EBIVALORMON no tipo Float

In [ ]:
#7395.18 Valida a correspondência entre os campos CMPID e TPMOID
mod_relacao = set(arq_relacao['relacao'].astype('str'))
relacao = [df_main['CMPID'] + df_main['TPMOID']]
for itemrelacao in relacao:
    relacao2 = set(itemrelacao)
    for itemrelacao2 in relacao2:
        if itemrelacao2 not in mod_relacao:
            print('ERRO!! A relação ',itemrelacao2,' (CMPID/TPMOID) não é uma relação possivel.')
            critica7395_18 = ('Critica 7395.18, A relação ',itemrelacao2,' (CMPID/TPMOID) não é uma relação possivel.')
            criticas_impeditivas.append(critica7395_18)
        else:
            print('A relação ',itemrelacao2,' (CMPID/TPMOID) é uma relação possivel.')
        continue

In [ ]:
criticas_impeditivas  #Lista das criticas impeditivas

In [ ]:
df_criticas_i = pd.DataFrame(criticas_impeditivas)     #criacao do DataFrame das criticas impeditivas
df_criticas_i = df_criticas_i.rename(columns={0:'Descrição'})
df_criticas_i

In [ ]:
df_criticas_i2 = pd.concat([dfc01,dfc02,dfc03,dfc04,dfc05,df13_1,df13_2,df13_3])
df_criticas_i2

In [ ]:
if criticas_impeditivas == []:                             # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 379')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 379')
    df_criticas_i.to_csv('Criticas_Impeditivas_379.csv')   # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_i2) == 0:                               # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 379')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 379')
    df_criticas_i2.to_csv('Criticas_Impeditivas_379_2.csv')   # Gerando csv das criticas impeditivas

# Criticas (Validacoes) Não Impeditivas

In [ ]:
#7395.16 Verifica se o campo EBICODCESS corresponde a um código de sociedade válido 
dadosebicodcess = set(df_main['EBICODCESS'])                               #Escolhendo a coluna dos dados
for itemebicodcess in dadosebicodcess: 
    if itemebicodcess not in entcod:
        print('ERRO!! EBICODCESS não corresponde a um código de sociedade válido', itemebicodcess)
        critica7395_16 = ('Critica 7395.16, EBICODCESS não corresponde a um código de sociedade válido', itemebicodcess)
        criticas.append(critica7395_16)
    else:
        print('EBICODCESS corresponde a um código de sociedade válido.', itemebicodcess)
        continue

In [ ]:
# Criando DataFrame copia com Campos CMPID, EBICODCESS, ENTCODIGO no formato int
df_mains = df_main.copy()
df_mains['CMPID'] = df_mains['CMPID'].astype('int64')
df_mains['EBICODCESS'] = df_mains['EBICODCESS'].astype('int64')
df_mains['ENTCODIGO'] = df_mains['ENTCODIGO'].astype('int64')

In [ ]:
#7395.17 Valida a correspondência entre os campos CMPID e EBICODCESS 
relacao17_38 = df_mains[(df_mains['CMPID'] == 1038) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao17_39 = df_mains[(df_mains['CMPID'] == 1039) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao17_42 = df_mains[(df_mains['CMPID'] == 1042) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao17_44 = df_mains[(df_mains['CMPID'] == 1044) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao17_47 = df_mains[(df_mains['CMPID'] == 1047) & (df_mains['EBICODCESS'].values != df_mains['ENTCODIGO'].values)]

In [ ]:
i = 34   #Definindo numero de colunas

# Adicionando criticas com CMPIDs 1038, 1039, 1042, 1044, 1047 na lista 'criticas'
if relacao17_38.shape != (0,i): 
    critica7395_17_38 = ('Critica 7395.17, O CMPID é 1038 e o EBICODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7395_17_38)
elif relacao17_39.shape != (0,i): 
    critica7392_17_39 = ('Critica 7395.17, O CMPID é 1039 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_39)
elif relacao17_42.shape != (0,i): 
    critica7392_17_42 = ('Critica 7395.17, O CMPID é 1042 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_42)
elif relacao17_44.shape != (0,i): 
    critica7392_17_44 = ('Critica 7395.17, O CMPID é 1044 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_44)
elif relacao17_47.shape != (0,i): 
    critica7392_17_47 = ('Critica 7395.17, O CMPID é 1047 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_47)
else: print('CMPIDs 1038, 1039, 1042, 1044, 1047. Valida campo EBICODCESS')

In [ ]:
#7395.17 Valida a correspondência entre os campos CMPID e EBICODCESS 
relacao17_40 = df_mains[(df_mains['CMPID'] == 1040) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)] 
relacao17_41 = df_mains[(df_mains['CMPID'] == 1041) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)] 
relacao17_45 = df_mains[(df_mains['CMPID'] == 1045) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)] 
relacao17_46 = df_mains[(df_mains['CMPID'] == 1046) & (df_mains['EBICODCESS'].values <= 1) & (df_mains['EBICODCESS'].values >= 9999)] 

In [ ]:
# Adicionando criticas com CMPIDs 1040, 1041, 1045, 1046 na lista 'criticas'
if relacao17_40.shape != (0,i): 
    critica7395_17_40 = ('Critica 7395.17, O CMPID é 1040 e o EBICODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7395_17_40)
elif relacao17_41.shape != (0,i): 
    critica7392_17_41 = ('Critica 7395.17, O CMPID é 1041 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_41)
elif relacao17_45.shape != (0,i): 
    critica7392_17_45 = ('Critica 7395.17, O CMPID é 1045 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_45)
elif relacao17_46.shape != (0,i): 
    critica7392_17_46 = ('Critica 7395.17, O CMPID é 1046 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_46)
else: print('CMPIDs 1040, 1041, 1045, 1046. Valida campo EBICODCESS')

In [ ]:
#7395.17 Valida a correspondência entre os campos CMPID e EBICODCESS 
relacao17_43 = df_mains[(df_mains['CMPID'] == 1043) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 
relacao17_48 = df_mains[(df_mains['CMPID'] == 1048) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 
relacao17_49 = df_mains[(df_mains['CMPID'] == 1049) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 
relacao17_50 = df_mains[(df_mains['CMPID'] == 1050) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)]
relacao17_51 = df_mains[(df_mains['CMPID'] == 1051) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 
relacao17_52 = df_mains[(df_mains['CMPID'] == 1052) & (df_mains['EBICODCESS'].values <= 30000) & (df_mains['EBICODCESS'].values >= 59999)] 

In [ ]:
# Adicionando criticas com CMPIDs 1043, 1048, 1049, 1050 ,1051 ,1052 na lista 'criticas'
if relacao17_43.shape != (0,i): 
    critica7395_17_43 = ('Critica 7395.17, O CMPID é 1043 e o EBICODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7395_17_43)
elif relacao17_48.shape != (0,i): 
    critica7392_17_48 = ('Critica 7395.17, O CMPID é 1048 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_48)
elif relacao17_49.shape != (0,i): 
    critica7392_17_49 = ('Critica 7395.17, O CMPID é 1049 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_49)
elif relacao17_50.shape != (0,i): 
    critica7392_17_50 = ('Critica 7395.17, O CMPID é 1050 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_50)
elif relacao17_51.shape != (0,i): 
    critica7392_17_51 = ('Critica 7395.17, O CMPID é 1051 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_51)
elif relacao17_52.shape != (0,i): 
    critica7392_17_52 = ('Critica 7395.17, O CMPID é 1052 e o EBICODCESS é diferente do ENTCODIGO')
    criticas.append(critica7392_17_52)
else: print('CMPIDs 1043, 1048, 1049, 1050 ,1051 ,1052. Valida campo EBICODCESS')

In [ ]:
#7395.20 Verifica se o CPF do participante (EBICPFPART) e do beneficiário (EBICPFBENF) são diferentes de zero 
for itemcpfp in cpfparticipante:
    if itemcpfp == '00000000000':
        print("Erro!! Critica 7395.20 o CPF ", itemcpfp, " não é um CPF valido.")
        critica7395_20 = ("Critica 7395.20","o CPF ", itemcpfp, " não é um CPF valido.")
        criticas.append(critica7395_20)

print('CPFs dos participantes nos campos (EBICPFPART) são validos')    

In [ ]:
for itemcpfb in cpfbeneficiario:
    if itemcpfb == '00000000000':
        print("Erro!! Critica 7395.20 o CPF ", itemcpfb, " não é um CPF valido.")
        critica7395_20 = ("Critica 7395.20","o CPF ", itemcpfb, " não é um CPF valido.")
        criticas.append(critica7395_20)

print('CPFs dos participantes nos campos (EBICPFBENF) são validos')  

In [ ]:
#7395.21 Verifica se a data do campo EBIDATAFIM é posterior à data do campo EBIDATAINICIO 
mask21 = (df_main['DTEBIDATAFIM']) <= (df_main['DTEBIDATAINICIO'])
df21 = df_main.loc[mask21]
if len(df21) > 0:
    df21.loc[:,'ID da Critica'] = ('CRITICA 7395.21')
df21

In [ ]:
#7395.22 Verifica se a data do campo EBIDATAINICIO é igual ou anterior à data do campo EBIDATAOCORR
mask22 = (df_main['DTEBIDATAINICIO']) > (df_main['DTEBIDATAOCORR'])
df22 = df_main.loc[mask22]
if len(df22) > 0:
    df22.loc[:,'ID da Critica'] = ('CRITICA 7395.22')
df22

In [ ]:
#7395.23 Verifica se a data do campo EBIDATAOCORR é igual ou anterior à data do campo EBIDATAFIM
mask23 = (df_main['DTEBIDATAOCORR']) > (df_main['DTEBIDATAFIM'])
df23 = df_main.loc[mask23]
if len(df23) > 0:
    df23.loc[:,'ID da Critica'] = ('CRITICA 7395.23')
df23

In [ ]:
#7395.24 Verifica se a data do campo EBIDATAOCORR é igual ou anterior à data do campo EBIDATACOMUNICA
mask24 = (df_main['DTEBIDATAOCORR']) > (df_main['DTEBIDATACOMUNICA'])
df24 = df_main.loc[mask24]
if len(df24) > 0:
    df24.loc[:,'ID da Critica'] = ('CRITICA 7395.24')
df24

In [ ]:
#7395.25 Verifica se a data do campo EBIDATACOMUNICA é igual ou anterior à data do campo EBIDATAREG
mask25 = (df_main['DTEBIDATACOMUNICA']) > (df_main['DTEBIDATAREG'])
df25 = df_main.loc[mask25]
if len(df25) > 0:
    df25.loc[:,'ID da Critica'] = ('CRITICA 7395.25')
df25

In [ ]:
#7395.26 Se o TPMOPREV for 0001, o TPMOID for 0001 e o CMPID for 1039, 1040, 1041, 1044, 1045 ou 1046, verifica se o mês e 
#ano da data do campo EBIDATAREG é igual ao mês e ano da data do campo MRFMESANO. Para os demais casos, verifica se a data dos 
#campos EBIDATAREG é igual ou anterior à data do campo MRFMESANO
mask26_rel_TT = (df_main['TPMOID'] == '0001') & (df_main['TPMOPREV'] == '0001')
mask26_cmpid = (df_main['CMPID'] == '1039') | (df_main['CMPID'] == '1040')| (df_main['CMPID'] == '1041')| (df_main['CMPID'] == '1044')| (df_main['CMPID'] == '1045')| (df_main['CMPID'] == '1046')
mask26_rel = (mask26_rel_TT) & (mask26_cmpid)
maskmr = df_main['DTMRFMESANO'].dt.strftime("%Y-%m")
maskreg = df_main['DTEBIDATAREG'].dt.strftime("%Y-%m")

mask26_1 = ((maskmr != maskreg)) & (mask26_rel)
df26_1 = df_main.loc[mask26_1]
if len(df26_1) > 0:
    df26_1.loc[:,'ID da Critica'] = ('CRITICA 7395.26')
df26_1

In [ ]:
mask26_TT_2 = (df_main['TPMOID'] != '0001') | (df_main['TPMOPREV'] != '0001')
mask26_cmpid_2=(df_main['CMPID'] == '1038') |(df_main['CMPID'] == '1047')|(df_main['CMPID'] == '1048')|(df_main['CMPID'] == '1049')|(df_main['CMPID'] == '1050')|(df_main['CMPID'] == '1051')|(df_main['CMPID'] == '1052')
mask26_rel_2 = (mask26_TT_2) & (mask26_cmpid_2)

mask26_2 = ((df_main['DTEBIDATAREG'] > df_main['DTMRFMESANO'])) != (mask26_rel_2)
df26_2 = df_main.loc[mask26_2]
if len(df26_2) > 0:
    df26_2.loc[:,'ID da Critica'] = ('CRITICA 7395.26')
df26_2

In [ ]:
#7395.27 Se o tipo de movimento (TPMOID) for 'aviso' 0001, a origem do movimento de regulação do evento (TPMOPREV) for 
#‘Planos de Risco (excluindo-se rendas vencidas)’ 0001 e o tipo de operação (CMPID) for 'direto - administrativo' 1039 ou 
#'cosseguro cedido - administrativo'1041 ou 'direto - judicial' 1044 ou 'cosseguro cedido - judicial' 1046, 
#verifica se a diferença de dias entre a data dos campos EBIDATACOMUNICA e EBIDATAREG é de até quinze dias
df_dates_comparacao.loc[:,'DTEBIDATACOMUNICA+15'] = df_main['DTMRFMESANO'] + timedelta(days = 15)

mask27_T = (df_main['TPMOID'] == '0001') & (df_main['TPMOPREV'] == '0001')

In [ ]:
mask27_1 = (mask27_T) & (df_main['CMPID'] == '1039')

mask27_DA = (mask27_1) & (df_dates_comparacao['DTEBIDATACOMUNICA+15'] < df_main['DTEBIDATAREG'])
df27_DA = df_main.loc[mask27_DA]
if len(df27_DA) > 0:
    df27_DA.loc[:,'ID da Critica'] = ('CRITICA 7395.27')
df27_DA

In [ ]:
mask27_2 = (mask27_T) & (df_main['CMPID'] == '1041')

mask27_CCA = (mask27_2) & (df_dates_comparacao['DTEBIDATACOMUNICA+15'] < df_main['DTEBIDATAREG'])
df27_CCA = df_main.loc[mask27_CCA]
if len(df27_CCA) > 0:
    df27_CCA.loc[:,'ID da Critica'] = ('CRITICA 7395.27')
df27_CCA

In [ ]:
mask27_3 = (mask27_T) & (df_main['CMPID'] == '1044')

mask27_DJ = (mask27_3) & (df_dates_comparacao['DTEBIDATACOMUNICA+15'] < df_main['DTEBIDATAREG'])
df27_DJ = df_main.loc[mask27_DJ]
if len(df27_DJ) > 0:
    df27_DJ.loc[:,'ID da Critica'] = ('CRITICA 7395.27')
df27_DJ

In [ ]:
mask27_4 = (mask27_T) & (df_main['CMPID'] == '1046')

mask27_CCJ = (mask27_4) & (df_dates_comparacao['DTEBIDATACOMUNICA+15'] < df_main['DTEBIDATAREG'])
df27_CCJ = df_main.loc[mask27_CCJ]
if len(df27_CCJ) > 0:
    df27_CCJ.loc[:,'ID da Critica'] = ('CRITICA 7395.27')
df27_CCJ

In [ ]:
#7395.28 Verifica se os campos EBIVALORPBAC e EBIVALORPBC são maiores ou iguais a zero para a combinação de TPMOID e TPMOPREV 
#(0001x0001; 0005x0001; 0006x0001) e se é zero para as demais combinações
mask28_rel_11 = (df_main['TPMOID'] == '0001') & (df_main['TPMOPREV'] == '0001')
mask28_rel_51 = (df_main['TPMOID'] == '0005') & (df_main['TPMOPREV'] == '0001')
mask28_rel_61 = (df_main['TPMOID'] == '0006') & (df_main['TPMOPREV'] == '0001')
mask28_rel = (mask28_rel_11) | (mask28_rel_51) | (mask28_rel_61)

mask28_PBAC = (df_main['fEBIVALORPBAC'] != 0.0) != (mask28_rel)
df28_PBAC = df_main.loc[mask28_PBAC]
if len(df28_PBAC) > 0:
    df28_PBAC.loc[:,'ID da Critica'] = ('CRITICA 7395.28')
df28_PBAC

In [ ]:
mask28_PBC = (df_main['fEBIVALORPBC'] != 0.0) != (mask28_rel)
df28_PBC = df_main.loc[mask28_PBC]
if len(df28_PBC) > 0:
    df28_PBC.loc[:,'ID da Critica'] = ('CRITICA 7395.28')
df28_PBC

In [ ]:
#7395.29 Verifica se o campo EBIVALORMOV é maior que zero quando o campo TPMOID for 0001, 0003, 0004, 0005 ou 0006
mask29_tpmoid = (df_main['TPMOID'] == '0001') | (df_main['TPMOID'] == '0003') | (df_main['TPMOID'] == '0004') | (df_main['TPMOID'] == '0005') | (df_main['TPMOID'] == '0006')
mask29 = (mask29_tpmoid) & (df_main['fEBIVALORMOV'] <= 0.0 )
df29 = df_main.loc[mask29]
if len(df29) > 0:
    df29.loc[:,'ID da Critica'] = ('CRITICA 7395.29')
df29

In [ ]:
#7395.30 Verifica se o campo EBIVALORMON é zero quando o campo TPMOID for 0003 e 0004 
mask30_tpmoid = (df_main['TPMOID'] == '0003') | (df_main['TPMOID'] == '0004')
mask30 = (mask30_tpmoid) & (df_main['fEBIVALORMON'] != 0.0 )
df30 = df_main.loc[mask30]
if len(df30) > 0:
    df30.loc[:,'ID da Critica'] = ('CRITICA 7395.30')
df30

In [ ]:
#7395.32 Verifica se os campos EBIVALORMOV e EBIVALORMON não são, simultaneamente, iguais a zero
mask32 = (df_main['fEBIVALORMOV'] == 0.0) & (df_main['fEBIVALORMON'] == 0.0)
df32 = df_main.loc[mask32]
if len(df32) > 0:
    df32.loc[:,'ID da Critica'] = ('CRITICA 7395.32')
df32

In [ ]:
#7395.33 Verifica se o campo EBINUMSIN é diferente de 0. 
mask33 = (df_main['EBINUMSIN'] == '00000000000000000000')
df33 = df_main.loc[mask33]
if len(df33) > 0:
    df33.loc[:,'ID da Critica'] = ('CRITICA 7395.33')
df33

In [ ]:
#7395.34 Verifica se o campo EBIBENVEN é zero para todas as combinações de TPMOID e TPMOPREV, exceto 0001x0001; 0005x0001; 0006x0001 
mask34_rel_11 = (df_main['TPMOID'] == '0001') & (df_main['TPMOPREV'] == '0001')
mask34_rel_51 = (df_main['TPMOID'] == '0005') & (df_main['TPMOPREV'] == '0001')
mask34_rel_61 = (df_main['TPMOID'] == '0006') & (df_main['TPMOPREV'] == '0001')
mask34_rel = (mask34_rel_11) | (mask34_rel_51) | (mask34_rel_61)

mask34 = (df_main['iEBIBENVEN'] != 0) != (mask34_rel)
df34 = df_main.loc[mask34]
if len(df34) > 0:
    df34.loc[:,'ID da Critica'] = ('CRITICA 7395.34')
df34

In [ ]:
################################################
# Adicionando Nome do Criador do output e data #
################################################

df_main.loc[:,'Criador'] = input("Insira seu nome: ") ### insere a coluna com o nome do criador da tabela
data_presente = datetime.now()
df_main.loc[:,'Data do output'] = data_presente ### insere a coluna com a data de criacao da tabela

In [ ]:
df_main

In [ ]:
criticas

In [ ]:
df_criticas = pd.DataFrame(criticas)
df_criticas = df_criticas.rename(columns={0:'Descrição'})
df_criticas

In [ ]:
df_criticas_2 = pd.concat([df21,df22,df23,df24,df25,df26_1,df26_2,df27_DA,df27_CCA,df27_DJ,df27_CCJ,df28_PBAC,df28_PBC,df29,df30,df32,df33,df34])
df_criticas_2

In [ ]:
if criticas == []:                   # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 379')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 379')
    df_criticas.to_csv('Criticas_nao_impeditivas379.csv')     # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_2) == 0:                             # Verificando se o DataFrame de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 379')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 379')
    df_criticas_2.to_csv('Criticas_nao_impeditivas379_2.csv')         # Gerando csv das criticas impeditivas

In [ ]:
df_main.to_csv('379_Trabalho.csv')